### Import Necessary Packages

In [216]:
# general
import os
import sys
import time
import random
import numpy as np
import pandas as pd
import polars as pl

# pytorch
import torch

# sklearn
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import VotingClassifier

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

import lightgbm as lgb
import catboost as cb
import xgboost as xgb

from tqdm.auto import tqdm

sys.path.append('../..')
from src.utils import load_env_vars

In [217]:
import warnings
warnings.filterwarnings("ignore")

### Device for inference

In [218]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {DEVICE} for inference')

Using cuda for inference


### Reproducibility

In [219]:
def seed_everything(seed : int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

### Global variables

In [220]:
DATA_DIR, _, _ = load_env_vars()

In [221]:
N_SPLITS = 5

In [222]:
SEED = 42
seed_everything(SEED)

### Data loading

In [223]:
train_meta_path = os.path.join(DATA_DIR, 'isic_2024/train-metadata.csv')
test_meta_path = os.path.join(DATA_DIR, 'isic_2024/test-metadata.csv')

In [224]:
train_metadata = pd.read_csv(train_meta_path)
test_metadata = pd.read_csv(test_meta_path)

### Columns

In [225]:
# Get the categorical & numerical columns
categorical_cols = test_metadata.select_dtypes(include=[object]).columns.tolist()
numerical_cols = test_metadata.select_dtypes(include=[np.number]).columns.tolist()

In [226]:
id_col, target_col, group_col = 'isic_id', 'target', 'patient_id'

irrelevant_cols = ['isic_id', 'patient_id', 'image_type', 'copyright_license', 'target']

# Filter out irrelevant columns
cat_features = [col for col in categorical_cols if col not in irrelevant_cols] + ['combined_anatomical_site']
num_features = [col for col in numerical_cols if col not in irrelevant_cols]

# Train columns
train_cols = cat_features + num_features

### Feature Engineering

In [227]:
def generate_features(metadata: pd.DataFrame):
    columns = []
    df = pl.DataFrame(metadata)

    df = df.with_columns([
        pl.col('tbp_lv_minorAxisMM').truediv(pl.col('clin_size_long_diam_mm'))
        .cast(pl.Float32).alias('lesion_size_ratio'),

        pl.col('tbp_lv_areaMM2').truediv(pl.col('tbp_lv_perimeterMM').pow(2))
        .cast(pl.Float32).alias('lesion_shape_index'),

        pl.col('tbp_lv_H').sub(pl.col('tbp_lv_Hext')).abs()
        .cast(pl.Float32).alias('hue_contrast'),

        pl.col('tbp_lv_L').sub(pl.col('tbp_lv_Lext')).abs()
        .cast(pl.Float32).alias('luminance_contrast'),

        pl.col('tbp_lv_deltaA').pow(2).add(pl.col('tbp_lv_deltaB').pow(2)).add(pl.col('tbp_lv_deltaL').pow(2)).sqrt()
        .cast(pl.Float32).alias('lesion_color_difference'),

        pl.col('tbp_lv_norm_border').add(pl.col('tbp_lv_symm_2axis'))
        .cast(pl.Float32).alias('border_complexity'),

        pl.col('tbp_lv_color_std_mean').truediv(pl.col('tbp_lv_radial_color_std_max'))
        .cast(pl.Float32).alias('color_uniformity'),
    ])
    columns += ['lesion_size_ratio', 'lesion_shape_index', 'hue_contrast', 'luminance_contrast', 'lesion_color_difference', 'border_complexity', 'color_uniformity']

    df = df.with_columns([
        pl.col('tbp_lv_x').pow(2).add(pl.col('tbp_lv_y').pow(2)).add(pl.col('tbp_lv_z').pow(2)).sqrt()
        .cast(pl.Float32).alias('position_distance_3d'),

        pl.col('tbp_lv_perimeterMM').truediv(pl.col('tbp_lv_areaMM2'))
        .cast(pl.Float32).alias('perimeter_to_area_ratio'),

        pl.col('tbp_lv_areaMM2').truediv(pl.col('tbp_lv_perimeterMM'))
        .cast(pl.Float32).alias('area_to_perimeter_ratio'),

        pl.col('tbp_lv_deltaLBnorm').add(pl.col('tbp_lv_norm_color'))
        .cast(pl.Float32).alias('lesion_visibility_score'),

        pl.col('anatom_site_general').add('_').add(pl.col('tbp_lv_location'))
        .cast(pl.Categorical).alias('combined_anatomical_site'),

        pl.col('tbp_lv_symm_2axis').mul(pl.col('tbp_lv_norm_border'))
        .cast(pl.Float32).alias('symmetry_border_consistency'),

        pl.col('tbp_lv_symm_2axis').mul(pl.col('tbp_lv_norm_border')).truediv(pl.col('tbp_lv_symm_2axis').add(pl.col('tbp_lv_norm_border')))
        .cast(pl.Float32).alias('consistency_symmetry_border'),
    ])
    columns += ['position_distance_3d', 'perimeter_to_area_ratio', 'area_to_perimeter_ratio', 'lesion_visibility_score', 'symmetry_border_consistency', 'consistency_symmetry_border']

    df = df.with_columns([
        pl.col('tbp_lv_stdL').truediv(pl.col('tbp_lv_Lext'))
        .cast(pl.Float32).alias('color_consistency'),

        pl.col('tbp_lv_stdL').mul(pl.col('tbp_lv_Lext')).truediv(pl.col('tbp_lv_stdL').add(pl.col('tbp_lv_Lext')))
        .cast(pl.Float32).alias('consistency_color'),

        pl.col('clin_size_long_diam_mm').mul(pl.col('age_approx'))
        .cast(pl.Float32).alias('size_age_interaction'),

        pl.col('tbp_lv_H').mul(pl.col('tbp_lv_color_std_mean'))
        .cast(pl.Float32).alias('hue_color_std_interaction'),

        pl.col('tbp_lv_norm_border').add(pl.col('tbp_lv_norm_color')).add(pl.col('tbp_lv_eccentricity')).truediv(3)
        .cast(pl.Float32).alias('lesion_severity_index'),

        pl.col('border_complexity').add(pl.col('lesion_shape_index'))
        .cast(pl.Float32).alias('shape_complexity_index'),

        pl.col('tbp_lv_deltaA').add(pl.col('tbp_lv_deltaB')).add(pl.col('tbp_lv_deltaL')).add(pl.col('tbp_lv_deltaLBnorm'))
        .cast(pl.Float32).alias('color_contrast_index'),
    ])
    columns += ['color_consistency', 'consistency_color', 'size_age_interaction', 'hue_color_std_interaction', 'lesion_severity_index', 'shape_complexity_index', 'color_contrast_index']

    df = df.with_columns([
        pl.col('tbp_lv_areaMM2').log1p()
        .cast(pl.Float32).alias('log_lesion_area'),

        pl.col('clin_size_long_diam_mm').truediv(pl.col('age_approx'))
        .cast(pl.Float32).alias('normalized_lesion_size'),

        pl.col('tbp_lv_H').add(pl.col('tbp_lv_Hext')).truediv(2)
        .cast(pl.Float32).alias('mean_hue_difference'),

        pl.col('tbp_lv_deltaA').pow(2).add(pl.col('tbp_lv_deltaB').pow(2)).add(pl.col('tbp_lv_deltaL').pow(2)).truediv(3).sqrt()
        .cast(pl.Float32).alias('std_dev_contrast'),

        pl.col('tbp_lv_color_std_mean').add(pl.col('tbp_lv_area_perim_ratio')).add(pl.col('tbp_lv_symm_2axis')).truediv(3)
        .cast(pl.Float32).alias('color_shape_composite_index'),

        pl.arctan2(pl.col('tbp_lv_x'), 
                   
        pl.col('tbp_lv_y'))
        .cast(pl.Float32).alias('lesion_orientation_3d'),

        pl.col('tbp_lv_deltaA').add(pl.col('tbp_lv_deltaB')).add(pl.col('tbp_lv_deltaL')).truediv(3)
        .cast(pl.Float32).alias('overall_color_difference'),
    ])
    columns += ['log_lesion_area', 'normalized_lesion_size', 'mean_hue_difference', 'std_dev_contrast', 'color_shape_composite_index', 'lesion_orientation_3d', 'overall_color_difference']

    df = df.with_columns([
        pl.col('tbp_lv_symm_2axis').mul(pl.col('tbp_lv_perimeterMM'))
        .cast(pl.Float32).alias('symmetry_perimeter_interaction'),

        pl.col('tbp_lv_area_perim_ratio').add(pl.col('tbp_lv_eccentricity')).add(pl.col('tbp_lv_norm_color')).add(pl.col('tbp_lv_symm_2axis')).truediv(4)
        .cast(pl.Float32).alias('comprehensive_lesion_index'),

        pl.col('tbp_lv_color_std_mean').truediv(pl.col('tbp_lv_stdLExt'))
        .cast(pl.Float32).alias('color_variance_ratio'),

        pl.col('tbp_lv_norm_border').mul(pl.col('tbp_lv_norm_color'))
        .cast(pl.Float32).alias('border_color_interaction'),

        pl.col('tbp_lv_norm_border').mul(pl.col('tbp_lv_norm_color')).truediv(pl.col('tbp_lv_norm_border').add(pl.col('tbp_lv_norm_color')))
        .cast(pl.Float32).alias('border_color_interaction_2'),

        pl.col('clin_size_long_diam_mm').truediv(pl.col('tbp_lv_deltaLBnorm'))
        .cast(pl.Float32).alias('size_color_contrast_ratio'),

        pl.col('tbp_lv_nevi_confidence').truediv(pl.col('age_approx'))
        .cast(pl.Float32).alias('age_normalized_nevi_confidence'),

        (pl.col('clin_size_long_diam_mm').pow(2).add(pl.col('age_approx').pow(2))).sqrt()
        .cast(pl.Float32).alias('age_normalized_nevi_confidence_2'),

        pl.col('tbp_lv_radial_color_std_max').mul(pl.col('tbp_lv_symm_2axis'))
        .cast(pl.Float32).alias('color_asymmetry_index'),
    ])
    columns += ['symmetry_perimeter_interaction', 'comprehensive_lesion_index', 'color_variance_ratio', 'border_color_interaction', 'border_color_interaction_2', 'size_color_contrast_ratio', 'age_normalized_nevi_confidence', 'age_normalized_nevi_confidence_2', 'color_asymmetry_index']

    df = df.with_columns([
        pl.col('tbp_lv_areaMM2').mul(pl.col('tbp_lv_x').pow(2).add(pl.col('tbp_lv_y').pow(2)).add(pl.col('tbp_lv_z').pow(2)).sqrt())
        .cast(pl.Float32).alias('volume_approximation_3d'),

        pl.col('tbp_lv_L').sub(pl.col('tbp_lv_Lext')).abs().add(pl.col('tbp_lv_A').sub(pl.col('tbp_lv_Aext')).abs())
        .add(pl.col('tbp_lv_B').sub(pl.col('tbp_lv_Bext')).abs())
        .cast(pl.Float32).alias('color_range'),

        pl.col('tbp_lv_eccentricity').mul(pl.col('tbp_lv_color_std_mean'))
        .cast(pl.Float32).alias('shape_color_consistency'),

        pl.col('tbp_lv_perimeterMM').truediv(pl.lit(2).mul(np.pi).mul(pl.col('tbp_lv_areaMM2').truediv(np.pi).sqrt()))
        .cast(pl.Float32).alias('border_length_ratio'),

        pl.col('age_approx').mul(pl.col('clin_size_long_diam_mm')).mul(pl.col('tbp_lv_symm_2axis'))
        .cast(pl.Float32).alias('age_size_symmetry_index'),

        pl.col('age_approx').mul(pl.col('tbp_lv_areaMM2')).mul(pl.col('tbp_lv_symm_2axis'))
        .cast(pl.Float32).alias('index_age_size_symmetry'),
            
        pl.col('isic_id').count().over('patient_id')
        .cast(pl.Int16).alias('count_per_patient'),
    ])
    columns += ['volume_approximation_3d', 'color_range', 'shape_color_consistency', 'border_length_ratio', 'age_size_symmetry_index', 'index_age_size_symmetry', 'count_per_patient']

    # Apply z-score aggregation to all numeric (float dtype) columns: z-score = (x - mean) / std
    z_score = [
        pl.col(col).sub(pl.col(col).mean()).truediv(pl.col(col).std()).over('patient_id')
        .cast(pl.Float32).alias(f'{col}_zscore') for col in df.columns if df[col].dtype == pl.Float32
        ]
    
    columns += [f'{col}_zscore' for col in df.columns if df[col].dtype == pl.Float32]
    df = df.with_columns(z_score)

    df = df.to_pandas()
    
    return df, columns

### Encoding categorical variables

In [228]:
def encode_categorical(df_train, df_test, train_cols):
    global cat_features
    
    encoder = OneHotEncoder(sparse_output=False, dtype=np.int32, handle_unknown='ignore')
    encoder.fit(df_train[cat_features])
    
    new_cat_features = [f'onehot_{i}' for i in range(len(encoder.get_feature_names_out()))]

    df_train[new_cat_features] = encoder.transform(df_train[cat_features])
    df_train[new_cat_features] = df_train[new_cat_features].astype('category')

    df_test[new_cat_features] = encoder.transform(df_test[cat_features])
    df_test[new_cat_features] = df_test[new_cat_features].astype('category')

    for col in cat_features:
        train_cols.remove(col)

    train_cols.extend(new_cat_features)
    cat_features = new_cat_features
    
    return df_train, df_test

### Apply Encoding & Feature Engineering

In [229]:
train_meta, new_num_columns = generate_features(train_metadata)
test_meta, _ = generate_features(test_metadata)
train_cols += new_num_columns

In [230]:
train_meta, test_meta = encode_categorical(train_meta, test_meta, train_cols)

### Training

#### LightGBM

In [231]:
lgb_params = {
    'objective':        'binary',
    'verbosity':        -1,
    'n_iter':           200,
    'boosting_type':    'gbdt',
    'random_state':     SEED,
    'lambda_l1':        0.08758718919397321, 
    'lambda_l2':        0.0039689175176025465, 
    'learning_rate':    0.03231007103195577, 
    'max_depth':        4, 
    'num_leaves':       103, 
    'colsample_bytree': 0.8329551585827726, 
    'colsample_bynode': 0.4025961355653304, 
    'bagging_fraction': 0.7738954452473223, 
    'bagging_freq':     4, 
    'min_data_in_leaf': 85, 
    'scale_pos_weight': 2.7984184778875543,
}

lgb_model = Pipeline([
    ('classifier', lgb.LGBMClassifier(**lgb_params)),
])

##### CatBoost

In [232]:
cb_params = {
    'loss_function':     'Logloss',
    'iterations':        200,
    'verbose':           False,
    'random_state':      SEED,
    'max_depth':         7, 
    'learning_rate':     0.06936242010150652, 
    'scale_pos_weight':  2.6149345838209532, 
    'l2_leaf_reg':       6.216113851699493, 
    'subsample':         0.6249261779711819, 
    'min_data_in_leaf':  24,
    'cat_features':      cat_features,
}

cb_model = Pipeline([
    ('classifier', cb.CatBoostClassifier(**cb_params)),
])

##### XGBoost

In [233]:
xgb_params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       SEED,
    'learning_rate':      0.08501257473292347, 
    'lambda':             8.879624125465703, 
    'alpha':              0.6779926606782505, 
    'max_depth':          6, 
    'subsample':          0.6012681388711075, 
    'colsample_bytree':   0.8437772277074493, 
    'colsample_bylevel':  0.5476090898823716, 
    'colsample_bynode':   0.9928601203635129, 
    'scale_pos_weight':   3.29440313334688,
}

xgb_model = Pipeline([
    ('classifier', xgb.XGBClassifier(**xgb_params)),
])

##### Voting Classifier

In [234]:
soft_estimator = VotingClassifier([
    ('lgb', lgb_model), ('cb', cb_model), ('xgb', xgb_model),
], voting='soft')

##### Partial AUC

In [235]:
def custom_metric(estimator, X, y_true):
    y_hat = estimator.predict_proba(X)[:, 1]
    min_tpr = 0.80
    max_fpr = abs(1 - min_tpr)
    
    v_gt = abs(y_true - 1)
    v_pred = np.array([1.0 - x for x in y_hat])
    
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return partial_auc

##### Cross-Validation

In [236]:
sgkf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

##### Training function

In [237]:
def get_train_groups(df, group_split = 10):
    train_groups = []

    # Get the positive and negative samples
    pos_samples = df[df.target == 1].reset_index(drop=True)
    neg_samples = df[df.target == 0].reset_index(drop=True)

    # Split the negative samples into 10 groups
    neg_samples['group'] = neg_samples.index % group_split
    
    # Concatenate the positive samples with the negative samples
    for i in range(group_split):
        train_group = pd.concat([pos_samples, neg_samples[neg_samples['group'] == i]]).reset_index(drop=True)
        train_groups.append(train_group)

    return train_groups

In [238]:
def train(metadata, train_cols, cv):

    X = metadata[train_cols]
    y = metadata[target_col]
    groups = metadata[group_col]

    models = {
        'fold1': [],
        'fold2': [],
        'fold3': [],
        'fold4': [],
        'fold5': []
    }

    scores = {
        'fold1': [],
        'fold2': [],
        'fold3': [],
        'fold4': [],
        'fold5': []
    }

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y, groups)):

        train_df = metadata.loc[train_idx].reset_index(drop=True)
        valid_df = metadata.loc[valid_idx].reset_index(drop=True)

        train_groups = get_train_groups(train_df)

        for i, train_group in enumerate(tqdm(train_groups, desc=f'Fold {fold+1}')):

            X_train, X_valid = train_group[train_cols], valid_df[train_cols]
            y_train, y_valid = train_group[target_col], valid_df[target_col]

            model = soft_estimator.fit(X_train, y_train)

            score = custom_metric(model, X_valid, y_valid)
        
            models[f'fold{fold+1}'].append(model)
            scores[f'fold{fold+1}'].append(score)

        print(f'Scores: {scores[f"fold{fold+1}"]}')
        print(f'Fold {fold+1} - Mean PAUC: {np.mean(scores[f"fold{fold+1}"])}')

    return models, scores    

In [239]:
models, scores = train(train_meta, train_cols, sgkf)

Fold 1:   0%|          | 0/10 [00:00<?, ?it/s]

XGBoostError: [21:17:20] /workspace/src/data/../common/../data/gradient_index.h:94: Check failed: valid: Input data contains `inf` or a value too large, while `missing` is not set to `inf`
Stack trace:
  [bt] (0) /home/bracs/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x22dbbc) [0x7f013d42dbbc]
  [bt] (1) /home/bracs/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x52daae) [0x7f013d72daae]
  [bt] (2) /home/bracs/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x51b5fd) [0x7f013d71b5fd]
  [bt] (3) /home/bracs/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x51d82c) [0x7f013d71d82c]
  [bt] (4) /home/bracs/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x4cce3a) [0x7f013d6cce3a]
  [bt] (5) /home/bracs/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(XGQuantileDMatrixCreateFromCallback+0x18c) [0x7f013d34546c]
  [bt] (6) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7f02602d7e2e]
  [bt] (7) /lib/x

In [ ]:
print(f'Average PAUC: {np.mean([np.mean(scores[f"fold{i+1}"]) for i in range(N_SPLITS)])}')